In [ ]:
#!pip install -q pytorch-lightning

In [ ]:
conda install pytorch torchvision -c pytorch

In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl

In [ ]:
df_train= pd.read_csv('../input/driving-behavior/train_motion_data.csv')
df_test= pd.read_csv('test_motion_data.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
df_train.columns

In [ ]:
df_test.head()

In [ ]:
df_test.describe()

In [ ]:
df_test.columns

In [ ]:
set(df_test.columns).difference(set(df_train.columns)) #Check for additional columns

In [ ]:
df_train.Class.unique()

In [12]:
col_dict= {'SLOW':-0.5,'NORMAL':0,'AGGRESSIVE':0.5}
df_train_new= df_train.replace({'Class':col_dict})
df_test_new= df_test.replace({'Class':col_dict})
df_train_new.head()

,AccX,AccY,AccZ,GyroX,GyroY,GyroZ,Class,Timestamp
0,0.000000,0.000000,0.000000,0.059407,-0.174707,0.101938,0.0,3581629
1,-1.624864,-1.082492,-0.204183,-0.028558,0.051313,0.135536,0.0,3581630
2,-0.594660,-0.122410,0.220502,-0.019395,-0.029322,0.087888,0.0,3581630
3,0.738478,-0.228456,0.667732,0.069791,-0.029932,0.054902,0.0,3581631
4,0.101741,0.777568,-0.066730,0.030696,-0.003665,0.054902,0.0,3581631


In [13]:
df_train_new=df_train_new.drop('Timestamp',axis=1)
df_test_new=df_test_new.drop('Timestamp',axis=1)

In [14]:
df_train_new

,AccX,AccY,AccZ,GyroX,GyroY,GyroZ,Class
0,0.000000,0.000000,0.000000,0.059407,-0.174707,0.101938,0.0
1,-1.624864,-1.082492,-0.204183,-0.028558,0.051313,0.135536,0.0
2,-0.594660,-0.122410,0.220502,-0.019395,-0.029322,0.087888,0.0
3,0.738478,-0.228456,0.667732,0.069791,-0.029932,0.054902,0.0
4,0.101741,0.777568,-0.066730,0.030696,-0.003665,0.054902,0.0
...,...,...,...,...,...,...,...
3639,0.915688,-2.017489,1.687505,0.450360,0.384845,-1.236468,-0.5
3640,-1.934203,0.914925,-0.096013,0.321468,0.649350,-0.477162,-0.5
3641,-0.222845,0.747304,-0.887430,0.361174,-0.406836,0.054291,-0.5
3642,-0.349423,0.067261,0.394368,-0.132405,0.020159,-0.004963,-0.5


In [15]:
class DrivingDataset(Dataset):
    def __init__(self,df):
        self.X= df.iloc[:,:-1].values
        self.y= df.iloc[:,-1].values
    def __len__(self):
        return len(self.X)
    def __getitem__(self,idx):
        return torch.tensor(self.X[idx]).float(),torch.tensor(self.y[idx]).float()
    
train_dataset= DrivingDataset(df_train_new)
test_dataset= DrivingDataset(df_test_new)

In [16]:
train_loader= DataLoader(train_dataset,batch_size=32,shuffle=True,num_workers=4)
test_loader= DataLoader(test_dataset,batch_size=64,num_workers=4)

In [17]:
class DrivingNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1= nn.Linear(6,3)
        self.lin2= nn.Linear(3,2)
        self.lin3= nn.Linear(2,1)
    def forward(self,x):
        return torch.sigmoid(self.lin3(torch.relu(self.lin2(torch.relu(self.lin1(x))))))
model=DrivingNetwork()
model
    

DrivingNetwork(
  (lin1): Linear(in_features=6, out_features=3, bias=True)
  (lin2): Linear(in_features=3, out_features=2, bias=True)
  (lin3): Linear(in_features=2, out_features=1, bias=True)
)

In [18]:
n_epochs=15
optimizer= optim.RMSprop(model.parameters(),lr=1e-3)
loss_fn= nn.BCEWithLogitsLoss()

for epoch in range(1,n_epochs+1):
    model.train()
    batch=0
    for X,y in train_loader:
        z= model(X)
        loss= loss_fn(z,y.unsqueeze(1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch+=1
        loss_out= loss.item()/batch
        
    if epoch==1 or epoch%2==0:
        print("Epoch: %d TrainLoss: %f"%(epoch,loss_out))
        
    model.eval()
    batch=0
    for x,y in test_loader:
        z= model(x)
        loss= loss_fn(z,y.unsqueeze(1))
        
        batch+=1
        loss_out= loss.item()/batch
    if epoch==1 or epoch%2==0:
        print("Epoch: %d TestLoss: %f"%(epoch,loss_out))
        
        
    


Epoch: 1 TrainLoss: 0.007701
Epoch: 1 TestLoss: 0.021781
Epoch: 2 TrainLoss: 0.007245
Epoch: 2 TestLoss: 0.018035
Epoch: 4 TrainLoss: 0.006363
Epoch: 4 TestLoss: 0.014921
Epoch: 6 TrainLoss: 0.006161
Epoch: 6 TestLoss: 0.014311
Epoch: 8 TrainLoss: 0.006100
Epoch: 8 TestLoss: 0.014186
Epoch: 10 TrainLoss: 0.006084
Epoch: 10 TestLoss: 0.014157
Epoch: 12 TrainLoss: 0.006082
Epoch: 12 TestLoss: 0.014149
Epoch: 14 TrainLoss: 0.006081
Epoch: 14 TestLoss: 0.014147


In [19]:
class DrivingTrainer(pl.LightningModule):
    def __init__(self,lr=1e-4):
        super().__init__()
        self.save_hyperparameters()
        self.lr=lr
        self.model=DrivingNetwork()
    def configure_optimizers(self):
        return optim.RMSprop(self.model.parameters(),lr=self.lr)
    def training_step(self,batch,idx):
        x,y= batch
        z= self.model(x)
        loss= nn.BCEWithLogitsLoss()(z,y.unsqueeze(1))
        self.log('train-loss',loss)
        return loss
    def validation_step(self,batch,idx):
        x,y= batch
        z= self.model(x)
        loss= nn.BCEWithLogitsLoss()(z,y.unsqueeze(1))
        self.log('val-loss',loss)
        return loss
        

In [20]:
model= DrivingTrainer()
trainer= pl.Trainer(max_epochs=25)
trainer.fit(model,train_loader,test_loader)

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]